In [2]:
from urllib import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import string
from lxml import html



In [3]:
#Subject, Subject Code
file_list = pd.Series(['E','GEO','GOV','SSS','US','WH'], index=['Economics','Geography','Government','Social Studies Skills','US History', 'World History'])
#    data=[['Economics', 'E'], 
#          ['Geography', 'GEO'], 
#          ['Government', 'GOV'], 
#          ['Social Studies Skills', 'SSS'], 
#          ['US History', 'US'], 
#          ['World History', 'WH']],
#          columns=['Subject', 'Subject_Code'])
#print file_list
#subject = file_list.iloc[0,0]
#subject_code = file_list.iloc[0,1]
#standards_list = []
keys = ['Subject','Parent Text', 'Parent TID', 'Child Text', 'Child TID', 'Standard Code']

filelist_2= pd.DataFrame(data=[['Economics', 'E'], 
          ['Geography', 'GEO'], 
          ['Government', 'GOV'], 
          ['Social Studies Skills', 'SSS'], 
          ['US History', 'US'], 
          ['World History', 'WH']],
          columns=['Subject', 'Subject_Code'])

In [8]:
#second_parent_children = second_parent.ul.findAll("a")
#print second_parent_children

def scrape_standards(standards_list, Subject, Subject_Code):
    filename = r'file:///Users\Melissa\Documents\Udacity\Untitled Folder\High School ' + subject + '.htm'
    file = urlopen(filename)
    soup = BeautifulSoup(file, "lxml")
    standards = soup.ul
    parent = standards.li
    parent_code = 1
    
    while parent != None:
        parent_text = parent.a.contents
        print parent_text
        parent_tid = parent.a['href'][-7:]
        #print parent
        children = parent.ul.findChildren(recursive=False)
        standard_code = subject_code + '.' + str(parent_code)
        child_code = 1
        standards_list.append(dict(zip(keys, [subject, parent_text, parent_tid, '', '', standard_code])))    
        #loop through children
        for child in children:
            child_text = child.a.contents
            child_tid = child.a['href'][-7:]
            standard_code = subject_code + '.' + str(parent_code) + '.' + str(child_code)
            standards_list.append(dict(zip(keys, [subject, parent_text, parent_tid, child_text, child_tid, standard_code])))    
            child_code += 1
        parent = parent.next_sibling
        print parent
        parent_code += 1
    return(standards_list)

In [5]:
#temp = filelist_2.apply(scrape_standards, axis = 1, args=())  --- doesn't work

In [9]:
row = 0
standards_list=[]
while row < 6:
    subject = filelist_2.iloc[row,0]
    subject_code = filelist_2.iloc[row,1]
    scrape_standards(standards_list, subject, subject_code)
    row += 1

[u'Economic Components']
<li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=3938542">Economic Concepts</a><ul><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703846">Business Cycle</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703830">Choice</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=4128513">Competition</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703823">Economic Development</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703840">Economic Indicators</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703833">Economic Policy / Government Services</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703847">Entrepreneurship</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703842">Fiscal and Monetary Policy</a></li><li><a class="tree" href="/edit.epl?standard_

In [11]:
standards_list_df = pd.DataFrame(standards_list)
print standards_list_df

    Child TID                                 Child Text Parent TID  \
0                                                           3938544   
1     4128511                              [Agriculture]    3938544   
2     2703845                    [Economic Institutions]    3938544   
3     2703827                         [Economic Systems]    3938544   
4     2703828                    [Factors of Production]    3938544   
5     2703844                   [Financial Institutions]    3938544   
6     2703826                              [Free Market]    3938544   
7     4128512                                    [Labor]    3938544   
8     2703843                       [Medium of Exchange]    3938544   
9     2703841                                    [Taxes]    3938544   
10    2703835                               [Technology]    3938544   
11                                                          3938542   
12    2703846                           [Business Cycle]    3938542   
13    

In [12]:
#output to Excel
output = "Social Studies Taxonomy Export.xlsx"

writer = pd.ExcelWriter(output)
standards_list_df.to_excel(writer, sheet_name='Sheet1')
writer.save()